In [318]:
import pandas as pd
import numpy as np
import overpy
import overpass
import folium
from osmapi import OsmApi
import math



In [319]:
#Functions
def getDistance(long1, lat1, long2, lat2): #haversine
    r = 6371000 #radio terrestre medio, en metros 
    c = math.pi/180 #constante para transformar grados en radianes 
    return 2*r*math.asin(math.sqrt(math.sin(c*(lat2-lat1)/2)**2 + math.cos(c*lat1)*math.cos(c*lat2)*math.sin(c*(long2-long1)/2)**2))

def utmToLatLng(zone, easting, northing, northernHemisphere=True):
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3

    return (latitude, longitude)
#---------------------------------------------------------------


def getDataOfCsv(name):
    allData = None
    try:
       allData = pd.read_csv(name, encoding = "utf8", sep=';')
    except:
        allData = pd.read_csv(name, encoding = "ISO-8859-1", sep=';')
    return allData
#---------------------------------------------------------------


def getPointOfStreet(streetName, boundingBoxSearch):   
    apiOverPass = overpass.API()
    #sql = '(area[name="'+country+'"]->.b;rel(area.b)["name:es"="'+locationArea+'"];map_to_area -> .a;way[name="'+streetName+'"]'+str(boundingBoxSearch)+';'
    
    sql = 'way[name~"'+streetName+'"]'+str(boundingBoxSearch)+';'
  
    return apiOverPass.Get(sql)
#---------------------------------------------------------------


def fromPointsOfStretGetBestUbicationMinXY(pointsOfStreet, xtest, ytest):
    allCorrd = pointsOfStreet['features']
    minD = math.inf
    cx = 0.0
    cy = 0.0
    for geo in allCorrd:
        geometry = geo["geometry"]

        if geometry["type"].upper() == "LINESTRING":
            for c in geometry["coordinates"]:
                y = c[0]
                x = c[1]
                #txtHtml = "Coord: "+ str(c) 
                #print(x, ' - ', y)
                #poppin = folium.Popup(html=folium.element.IFrame(html=txtHtml,width=200,height=50))
                #folium.Marker([x,y], icon=folium.Icon(icon='glyphicon-plus', color='pink'),popup = poppin).add_to(markerCluster)
                d = getDistance(xtest, ytest, x, y)
                #print(d, ' --- ', str(c))
                if d < minD:  
                    #print(d, ' --- ', str(c))
                    cx = x
                    cy = y 
                    minD = d
    return cx,cy
#---------------------------------------------------------------    
    

In [320]:
#Load all data
anio = 2015
csvFile = 'ACCIDENTS_GU_BCN_' + str(anio) + str('.csv')
allData = getDataOfCsv(csvFile)
allData[:3]



,Número d'expedient,Codi districte,Nom districte,Codi barri,Nom barri,Codi carrer,Nom carrer,Num postal caption,Descripció dia setmana,Dia setmana,...,Dia de mes,Hora de dia,Descripció torn,Descripció causa vianant,Número de morts,Número de lesionats lleus,Número de lesionats greus,Número de vehicles implicats,Coordenada UTM (Y),Coordenada UTM (X)
0,2015S005807,-1,Desconegut,-1,Desconegut,-1,Desconegut,Desconegut,Dimarts,Dm,...,28,12,Matí,No és causa del vianant,0,2,0,2,-1,-1
1,2015S007685,10,Sant Martí,64,el Camp de l'Arpa del Clot,134801,Freser,0208 0208,Dimarts,Dm,...,13,19,Tarda,Desobeir el senyal del semàfor,0,1,0,1,"4585420,58","431779,16"
2,2015S001364,10,Sant Martí,64,el Camp de l'Arpa del Clot,161407,Indústria,0336 0336,Dissabte,Ds,...,21,21,Tarda,No és causa del vianant,0,1,0,1,"4585555,86","431913,65"


In [321]:
#Init all variables from location
#Barcelona zone = 31
zone = 31
country = "España"
locationArea = "Barcelona"
locationAreaBoundingBox = (41.3248770036,2.0520401001,41.4829908452,2.2813796997)



In [322]:
#CSV style columns detection
coordX = allData.columns.size - 1
coordY = allData.columns.size - 2
colDeath = 18
colMinor = 19
colSerious = 20

colDay = 14 
colMonth = 12
colYear = 11

colStreet = 6




In [323]:
#map creation

map = folium.Map(location=[41.388790,2.158990])

html="add <b>popup</b> here"
iframe = folium.element.IFrame(html=html,width=100,height=100)
poppin = folium.Popup(html="add <b>popup</b> here")

markerCluster = folium.MarkerCluster("Accidents").add_to(map)
clor = 'blue'


In [324]:
i = 0
for d in allData[550:600].iterrows():
    if d[1][1] != -1: #comprovacion invalid data
        X = float(d[1][coordX].replace(',','.'))
        Y = float(d[1][coordY].replace(',','.'))    
        l = utmToLatLng(zone,X,Y)  
        #print("Original: ", str(l))
        #icon=folium.Icon(icon='glyphicon-plus')
        #icon=folium.Icon(color='black')
        
        pointsLocat = getPointOfStreet(d[1][colStreet], locationAreaBoundingBox)
        
        newLoc = fromPointsOfStretGetBestUbicationMinXY(pointsLocat, l[0], l[1])
        nx = newLoc[0]
        ny = newLoc[1]
        
        #print(i, ' --> ', newLoc, " - ", l, ' -- ', d[1][colStreet])
        
        if int(d[1][colDeath]) > 0:
            clor = 'black'
        elif int(d[1][colSerious]) > 0:
            clor = 'red'
        else:
            clor = 'green'

        txtHtmlOld = "Id: "+ str(i) + ' - ' + str(l)     
        txtHtml = "Id: "+ str(i) + ' - ' + str(newLoc) 
        
        #txtHtml = "Dia: "+ str(d[1][colDay]) + '/' + str(d[1][colMonth]) + '/' + str(d[1][colYear])
        #txtHtml += '</br>' + "here"

        poppinOld = folium.Popup(html=folium.element.IFrame(html=txtHtml,width=200,height=50))
        poppin = folium.Popup(html=folium.element.IFrame(html=txtHtml,width=200,height=50))
        #folium.Marker([l[0],l[1]], icon=folium.Icon(icon='glyphicon-plus', color='blue'),popup = poppinOld).add_to(markerCluster)
        folium.Marker([nx,ny], icon=folium.Icon(icon='glyphicon-plus', color=clor),popup = poppin).add_to(markerCluster)
        i+=1

In [325]:
map

In [326]:
print(getDistance(41.41597824664133, 2.1807059053508873, 41.4145734, 2.1788221))
print(getDistance(41.41597824664133, 2.1807059053508873, 41.4138138, 2.1798863))

261.2361833071212
257.1898407380421


In [327]:
print(str(i))

50
